## Quick visualization of the stations


In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import geopandas as gpd
import folium as fl
# danish data: https://github.com/ok-dk/dagi
import pandas as pd

stations_vv = gpd.read_file("data/vejvejr_stations_ll.shp")
stations_vv.head(10)
m = fl.Map(zoom_start=100, layers_control=True, tiles="OpenStreetMap")
for _, r in stations_vv.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    geo_j.add_to(m)
m    





### load domain that includes north jutland's tip, which I selected using leafmap before


In [2]:
df_njl = gpd.read_file("data/DK/nordjylland.shp") # this one is the whole thing
df_njl = gpd.read_file("nordjylland.geojson")

df_kastrup = gpd.read_file("around_kastrup.geojson")
stations_vv.crs = {'init': 'EPSG:4326'}
#points_kastrup = gpd.sjoin(stations_vv, df_kastrup, op = 'within')
points_kastrup = gpd.sjoin(df_kastrup, stations_vv,op = 'contains')

points_kastrup.head(10)

,geometry,index_right
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",200
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",203
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",201
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",202
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",171
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",170
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",184
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",185
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",182
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",188


In [3]:
stations_vv.shape#.iloc[200]

(1041, 1)

In [4]:
ns=points_kastrup.shape[0]
print(f"There are {ns} stations in this area")

There are 42 stations in this area


In [6]:
m = fl.Map(zoom_start=100, layers_control=True, tiles="OpenStreetMap")
for _, r in points_kastrup.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    geo_j.add_to(m)
m    

Checking how many stations in total in the observation

In [7]:
import sqlite3
import pandas as pd
dbase="data/stations_coords_height.db"
con=sqlite3.connect(dbase)
com="SELECT * FROM roadstations"
df_stations = pd.read_sql(com,con)
con.close()

# read the observational data
OBS="/home/cap/Downloads/ROAD_MODEL_DATA/OBSTABLE_TROAD_2023.sqlite"

com="SELECT * FROM SYNOP"
con=sqlite3.connect(OBS)
df_obs_2023 = pd.read_sql(com,con)
con.close()

# read the forecast data from hour 00 or 12
FCST="/home/cap/Downloads/ROAD_MODEL_DATA/FCTABLE_TROAD_202301_00.sqlite"
FCST="/home/cap/Downloads/ROAD_MODEL_DATA/FCTABLE_TROAD_202301_12.sqlite"

com="SELECT * FROM FC"
con=sqlite3.connect(FCST)
df_fcst = pd.read_sql(com,con)
con.close()


In [8]:
# select only stations in a given month to compare with a previous year
df_obs_2023["datetime"]=pd.to_datetime(df_obs_2023["validdate"],unit="s")


In [9]:
df_obs_2023.head(10)
MONTH=1
LAST=31
YEAR=2023
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2301 = df_obs_2023[(df_obs_2023["datetime"] >= beg_month) & (df_obs_2023["datetime"] <= end_month)]

MONTH=2
LAST=28
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2302 = df_obs_2023[(df_obs_2023["datetime"] >= beg_month) & (df_obs_2023["datetime"] <= end_month)]

MONTH=3
LAST=31
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2303 = df_obs_2023[(df_obs_2023["datetime"] >= beg_month) & (df_obs_2023["datetime"] <= end_month)]



unique_ID_2301=obs_2301['SID'].unique()
unique_ID_2302=obs_2302['SID'].unique()
unique_ID_2303=obs_2303['SID'].unique()


In [10]:
obs_2301.sort_values(by="datetime")

,validdate,SID,lat,lon,elev,TROAD,datetime
0,1673402400,100001,55.995688,12.486580,58.560295,3.800000,2023-01-11 02:00:00
301823,1673402400,620601,56.963752,9.605396,2.519188,4.296300,2023-01-11 02:00:00
132442,1673402400,303000,55.032600,10.613100,39.375118,4.700000,2023-01-11 02:00:00
173114,1673402400,335100,55.402262,10.382715,20.900751,5.000000,2023-01-11 02:00:00
236497,1673402400,514200,56.437230,8.374210,12.104514,6.300000,2023-01-11 02:00:00
...,...,...,...,...,...,...,...
121555,1675123200,301400,55.175667,9.058456,34.947765,0.250305,2023-01-31 00:00:00
305229,1675123200,621100,57.030925,9.926267,2.904482,0.000000,2023-01-31 00:00:00
122038,1675123200,301500,55.098751,8.940486,21.745260,1.080590,2023-01-31 00:00:00
117708,1675123200,300700,55.420570,8.703890,8.627779,0.400000,2023-01-31 00:00:00


In [11]:
obs_2302.sort_values(by="datetime")

,validdate,SID,lat,lon,elev,TROAD,datetime
347158,1675209600,100001,55.995688,12.486580,58.560295,1.90000,2023-02-01
553954,1675209600,320200,55.166650,9.294880,44.343319,1.52015,2023-02-01
496256,1675209600,300000,55.625683,9.728276,11.608043,1.66667,2023-02-01
786198,1675209600,990700,55.332045,10.965891,0.089390,2.69231,2023-02-01
690965,1675209600,580000,56.297287,9.100365,50.025158,1.40000,2023-02-01
...,...,...,...,...,...,...,...
425093,1677542400,174200,55.510200,12.150240,15.633819,2.00106,2023-02-28
679961,1677542400,522200,55.955592,8.522484,7.265375,-1.10000,2023-02-28
679297,1677542400,522201,55.955621,8.522462,7.190556,-1.20000,2023-02-28
684584,1677542400,524500,56.141318,9.178030,64.439796,-2.70000,2023-02-28


In [14]:
obs_2303.sort_values(by="datetime")

,validdate,SID,lat,lon,elev,TROAD,datetime
821479,1677628800,100001,55.995688,12.486580,58.560295,-2.200000,2023-03-01
1081921,1677628800,342100,55.091840,10.651200,57.970062,0.274725,2023-03-01
1180021,1677628800,520300,55.969616,8.871111,27.668800,2.700000,2023-03-01
1312759,1677628800,401501,56.016365,9.882677,33.677418,-1.200000,2023-03-01
1180714,1677628800,520400,56.150396,8.905459,53.364185,2.000000,2023-03-01
...,...,...,...,...,...,...,...
1048767,1680220800,316100,55.549454,9.703747,19.710461,7.332110,2023-03-31
1049484,1680220800,320000,55.345281,9.264045,41.431263,6.941390,2023-03-31
1050200,1680220800,320100,55.269569,9.539442,40.314335,7.747250,2023-03-31
1051608,1680220800,320301,55.193851,9.513439,44.976276,7.100000,2023-03-31


In [15]:
ns = unique_ID_2301.shape[0]
print(f"There are {ns} stations in jan 2023")

ns = unique_ID_2302.shape[0]
print(f"There are {ns} stations in feb 2023")

ns = unique_ID_2303.shape[0]
print(f"There are {ns} stations in mar 2023")


There are 937 stations in jan 2023
There are 962 stations in feb 2023
There are 974 stations in mar 2023


In [16]:
# quick and dirty list I extracted from the glamtodel output

In [17]:
new_stations_id=[1241,1302,1305,1320,1389,1743,1901,1902,3011,3246,3422,3902,4000,4003,4004,4013,4101,4102,4104,4106,4111,4113,4114,4145,4147,4149,4162,4163,4164,4244,4249,4282,4283,4323,4325,4328,4329,4330,4331,4332,4334,4336,4900,5006,5007,5008,5109,5110,5201,6000,6245,6900,1382,1383,1384,1387,1388,1680,1681,2101,2103,2104,2105,2106,2107,2108,2112,2113,2115,2116,2400,3204,3205,3206,3208,3209,3210,3213,3214,3216,3217,3218,3219,3364,3367,3370,3371,3372,3373,3374,3375,3378,3379,3380,3381,3383,3384,3385,3386,3388,3501,3503,3504,4184,4185,4187,4189,4190,4191,5144,5146,5147,5148,6146,6148,6152,6228,6231,6232,6233,6234,6235,6236,6237,6238,6239,6427,6431,6432,6520,5167,
                6111,6209,6266,6267]

In [18]:
new_SID=[i*100 for i in new_stations_id]
new_SID.sort()
#new_SID[0:10]

[124100,
 130200,
 130500,
 132000,
 138200,
 138300,
 138400,
 138700,
 138800,
 138900]

In [19]:
obs_2301_includes_new = obs_2301[obs_2301["SID"].isin(new_SID)]

In [20]:
obs_2301_includes_new

,validdate,SID,lat,lon,elev,TROAD,datetime
176071,1673402400,337000,55.35735,10.045865,57.962009,5.3,2023-01-11 02:00:00
176072,1673456400,337000,55.35735,10.045865,57.962009,4.7,2023-01-11 17:00:00
176073,1673517600,337000,55.35735,10.045865,57.962009,6.0,2023-01-12 10:00:00
176074,1673575200,337000,55.35735,10.045865,57.962009,5.0,2023-01-13 02:00:00
176075,1673632800,337000,55.35735,10.045865,57.962009,5.9,2023-01-13 18:00:00
...,...,...,...,...,...,...,...
347000,1675112400,138700,55.74517,12.438650,35.310760,0.1,2023-01-30 21:00:00
347001,1675116000,138700,55.74517,12.438650,35.310760,-0.2,2023-01-30 22:00:00
347002,1675119600,138700,55.74517,12.438650,35.310760,-0.5,2023-01-30 23:00:00
347003,1675123200,138700,55.74517,12.438650,35.310760,-0.8,2023-01-31 00:00:00


In [21]:
obs_2301_not_new = obs_2301[~obs_2301["SID"].isin(new_SID)]

In [22]:
obs_2301_not_new

,validdate,SID,lat,lon,elev,TROAD,datetime
0,1673402400,100001,55.995688,12.486580,58.560295,3.80,2023-01-11 02:00:00
1,1673406000,100001,55.995688,12.486580,58.560295,3.90,2023-01-11 03:00:00
2,1673409600,100001,55.995688,12.486580,58.560295,4.00,2023-01-11 04:00:00
3,1673413200,100001,55.995688,12.486580,58.560295,4.20,2023-01-11 05:00:00
4,1673416800,100001,55.995688,12.486580,58.560295,4.80,2023-01-11 06:00:00
...,...,...,...,...,...,...,...
347133,1675108800,324700,55.112089,9.417246,46.957935,0.50,2023-01-30 20:00:00
347134,1675112400,324700,55.112089,9.417246,46.957935,0.25,2023-01-30 21:00:00
347135,1675116000,324700,55.112089,9.417246,46.957935,0.00,2023-01-30 22:00:00
347136,1675119600,324700,55.112089,9.417246,46.957935,-0.25,2023-01-30 23:00:00


Checking station data from other years when there was fewer stations

In [23]:
OBS="/home/cap/Downloads/ROAD_MODEL_DATA/OBSTABLE_TROAD_2022.sqlite"

com="SELECT * FROM SYNOP"
con=sqlite3.connect(OBS)
df_obs_2022 = pd.read_sql(com,con)
con.close()

In [24]:
df_obs_2022.columns

Index(['validdate', 'SID', 'lat', 'lon', 'elev', 'TROAD'], dtype='object')

In [25]:
df_obs_2022["datetime"]=pd.to_datetime(df_obs_2022["validdate"],unit="s")


In [26]:
df_obs_2022

,validdate,SID,lat,lon,elev,TROAD,datetime
0,1640984400,100001,55.995688,12.48658,58.560295,6.70,2021-12-31 21:00:00
1,1640988000,100001,55.995688,12.48658,58.560295,6.50,2021-12-31 22:00:00
2,1640991600,100001,55.995688,12.48658,58.560295,6.40,2021-12-31 23:00:00
3,1640995200,100001,55.995688,12.48658,58.560295,6.40,2022-01-01 00:00:00
4,1640998800,100001,55.995688,12.48658,58.560295,6.60,2022-01-01 01:00:00
...,...,...,...,...,...,...,...
1340341,1648749600,500800,56.258700,8.31666,19.815536,3.81,2022-03-31 18:00:00
1340342,1648753200,500800,56.258700,8.31666,19.815536,2.25,2022-03-31 19:00:00
1340343,1648756800,500800,56.258700,8.31666,19.815536,1.15,2022-03-31 20:00:00
1340344,1648760400,500800,56.258700,8.31666,19.815536,0.27,2022-03-31 21:00:00


In [27]:
MONTH=1
LAST=31
YEAR=2022
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2201 = df_obs_2022[(df_obs_2022["datetime"] >= beg_month) & (df_obs_2022["datetime"] <= end_month)]


In [28]:
obs_2201

,validdate,SID,lat,lon,elev,TROAD,datetime
3,1640995200,100001,55.995688,12.48658,58.560295,6.4,2022-01-01 00:00:00
4,1640998800,100001,55.995688,12.48658,58.560295,6.6,2022-01-01 01:00:00
5,1641002400,100001,55.995688,12.48658,58.560295,6.7,2022-01-01 02:00:00
6,1641006000,100001,55.995688,12.48658,58.560295,6.7,2022-01-01 03:00:00
7,1641009600,100001,55.995688,12.48658,58.560295,6.7,2022-01-01 04:00:00
...,...,...,...,...,...,...,...
505936,1643572800,426000,55.977800,10.18537,11.153507,0.9,2022-01-30 20:00:00
505937,1643576400,426000,55.977800,10.18537,11.153507,1.3,2022-01-30 21:00:00
505938,1643580000,426000,55.977800,10.18537,11.153507,1.6,2022-01-30 22:00:00
505939,1643583600,426000,55.977800,10.18537,11.153507,1.8,2022-01-30 23:00:00


In [29]:
MONTH=2
LAST=28
YEAR=2022
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2202 = df_obs_2022[(df_obs_2022["datetime"] >= beg_month) & (df_obs_2022["datetime"] <= end_month)]

In [30]:
unique_ID_2202=obs_2202['SID'].unique()


In [31]:
unique_ID_2302.shape

(962,)

In [32]:
unique_ID_2202.shape

(721,)

In [33]:
# Make all obs dataframes geopandas dataframes
geo_obs_2202 = gpd.GeoDataFrame(
    obs_2202, geometry=gpd.points_from_xy(obs_2202.lon, obs_2202.lat), crs="EPSG:4326"
)


In [34]:
geo_obs_2202

,validdate,SID,lat,lon,elev,TROAD,datetime,geometry
505977,1644390000,100001,55.995688,12.486580,58.560295,4.4,2022-02-09 07:00:00,POINT (12.48658 55.99569)
505978,1644393600,100001,55.995688,12.486580,58.560295,5.2,2022-02-09 08:00:00,POINT (12.48658 55.99569)
505979,1644397200,100001,55.995688,12.486580,58.560295,6.3,2022-02-09 09:00:00,POINT (12.48658 55.99569)
505980,1644400800,100001,55.995688,12.486580,58.560295,6.9,2022-02-09 10:00:00,POINT (12.48658 55.99569)
505981,1644408000,100001,55.995688,12.486580,58.560295,7.4,2022-02-09 12:00:00,POINT (12.48658 55.99569)
...,...,...,...,...,...,...,...,...
830267,1645992000,228000,54.939381,12.217004,8.970529,2.4,2022-02-27 20:00:00,POINT (12.21700 54.93938)
830268,1645995600,228000,54.939381,12.217004,8.970529,1.6,2022-02-27 21:00:00,POINT (12.21700 54.93938)
830269,1645999200,228000,54.939381,12.217004,8.970529,1.0,2022-02-27 22:00:00,POINT (12.21700 54.93938)
830270,1646002800,228000,54.939381,12.217004,8.970529,0.5,2022-02-27 23:00:00,POINT (12.21700 54.93938)


In [35]:
geo_obs_2302 = gpd.GeoDataFrame(
    obs_2302, geometry=gpd.points_from_xy(obs_2302.lon, obs_2302.lat), crs="EPSG:4326"
)


In [36]:
points_kastrup.head(10)

,geometry,index_right
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",200
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",203
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",201
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",202
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",171
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",170
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",184
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",185
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",182
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",188


In [37]:
geo_obs_2202

,validdate,SID,lat,lon,elev,TROAD,datetime,geometry
505977,1644390000,100001,55.995688,12.486580,58.560295,4.4,2022-02-09 07:00:00,POINT (12.48658 55.99569)
505978,1644393600,100001,55.995688,12.486580,58.560295,5.2,2022-02-09 08:00:00,POINT (12.48658 55.99569)
505979,1644397200,100001,55.995688,12.486580,58.560295,6.3,2022-02-09 09:00:00,POINT (12.48658 55.99569)
505980,1644400800,100001,55.995688,12.486580,58.560295,6.9,2022-02-09 10:00:00,POINT (12.48658 55.99569)
505981,1644408000,100001,55.995688,12.486580,58.560295,7.4,2022-02-09 12:00:00,POINT (12.48658 55.99569)
...,...,...,...,...,...,...,...,...
830267,1645992000,228000,54.939381,12.217004,8.970529,2.4,2022-02-27 20:00:00,POINT (12.21700 54.93938)
830268,1645995600,228000,54.939381,12.217004,8.970529,1.6,2022-02-27 21:00:00,POINT (12.21700 54.93938)
830269,1645999200,228000,54.939381,12.217004,8.970529,1.0,2022-02-27 22:00:00,POINT (12.21700 54.93938)
830270,1646002800,228000,54.939381,12.217004,8.970529,0.5,2022-02-27 23:00:00,POINT (12.21700 54.93938)


In [38]:
df_kas = points_kastrup.rename(columns={'index_right': 'other_name'})

match_kas_2202 = gpd.sjoin(df_kas,geo_obs_2202,op = 'contains')
match_kas_2302 = gpd.sjoin(df_kas,geo_obs_2302,op = 'contains')


In [39]:
match_kas_2202

,geometry,other_name,index_right,validdate,SID,lat,lon,elev,TROAD,datetime
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",200,515173,1644444000,101700,55.614113,12.510605,18.059750,3.0,2022-02-09 22:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",203,515173,1644444000,101700,55.614113,12.510605,18.059750,3.0,2022-02-09 22:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",201,515173,1644444000,101700,55.614113,12.510605,18.059750,3.0,2022-02-09 22:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",202,515173,1644444000,101700,55.614113,12.510605,18.059750,3.0,2022-02-09 22:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",171,515173,1644444000,101700,55.614113,12.510605,18.059750,3.0,2022-02-09 22:00:00
...,...,...,...,...,...,...,...,...,...,...
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",158,827208,1644555600,154301,55.689854,12.583619,6.305036,-1.6,2022-02-11 05:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",157,827208,1644555600,154301,55.689854,12.583619,6.305036,-1.6,2022-02-11 05:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",159,827208,1644555600,154301,55.689854,12.583619,6.305036,-1.6,2022-02-11 05:00:00
0,"POLYGON ((12.49151 55.67536, 12.63986 55.71252...",151,827208,1644555600,154301,55.689854,12.583619,6.305036,-1.6,2022-02-11 05:00:00


In [40]:
to_plot=match_kas_2202[match_kas_2202["datetime"] == datetime(2022,2,9,22)]

In [41]:
to_plot[["SID","lat","lon"]].drop_duplicates(subset=["SID"]).shape

(25, 3)

In [42]:
include_these=to_plot["SID"].to_list()
to_plot=geo_obs_2202[geo_obs_2202["SID"].isin(include_these)]

In [43]:
to_plot=to_plot.drop_duplicates(subset=["SID"])

In [44]:
to_plot.shape

(25, 8)

In [72]:
#for k,r in to_plot.iterrows():
#    print(r["SID"])
#

In [69]:
tooltip = "Click Here For Station name"

    
m = fl.Map(zoom_start=100, layers_control=True, tiles="OpenStreetMap")
for _, r in to_plot.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    station = r["SID"]
    this_lat = r["lat"]
    this_lon = r["lon"]
    marker = fl.Marker(
        location=[this_lat, this_lon],
        popup=f"<stong> {station}</stong>",
        tooltip=tooltip, permanent=True)
    marker.add_to(m)
    #geo_j.add_to(m)

for _, r in points_kastrup.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    geo_j.add_to(m)
m    

In [82]:
to_plot=match_kas_2302[match_kas_2302["datetime"] == datetime(2023,2,9,22)]
include_these=to_plot["SID"].to_list()
to_plot=geo_obs_2302[geo_obs_2302["SID"].isin(include_these)]
to_plot=to_plot.drop_duplicates(subset=["SID"])
to_plot.shape

(35, 8)

In [83]:
to_plot["SID"].to_list()

[101500,
 101501,
 101700,
 103500,
 103501,
 162000,
 162100,
 180000,
 180500,
 180600,
 180800,
 180900,
 181000,
 181100,
 181200,
 181400,
 181500,
 181700,
 181800,
 181900,
 182000,
 182100,
 182200,
 182300,
 182400,
 154600,
 154601,
 154900,
 154300,
 154301,
 154500,
 154501,
 154701,
 154700,
 154800]

In [85]:
m = fl.Map(zoom_start=100, layers_control=True, tiles="OpenStreetMap")
#for _, r in to_plot.iterrows():
#    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
#    geo_j = sim_geo.to_json()
#    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
#    geo_j.add_to(m)
for _, r in to_plot.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    station = r["SID"]
    this_lat = r["lat"]
    this_lon = r["lon"]
    marker = fl.Marker(
        location=[this_lat, this_lon],
        popup=f"<stong> {station}</stong>",
        tooltip=tooltip, permanent=True)
    marker.add_to(m)
    #geo_j.add_to(m)
  
for _, r in points_kastrup.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    geo_j.add_to(m)
m    

In [ ]:
# including data from 2021

In [73]:
OBS="/home/cap/Downloads/ROAD_MODEL_DATA/OBSTABLE_TROAD_2021.sqlite"

com="SELECT * FROM SYNOP"
con=sqlite3.connect(OBS)
df_obs_2021 = pd.read_sql(com,con)
con.close()
df_obs_2021["datetime"]=pd.to_datetime(df_obs_2021["validdate"],unit="s")

MONTH=1
LAST=31
YEAR=2021
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2101 = df_obs_2021[(df_obs_2021["datetime"] >= beg_month) & (df_obs_2021["datetime"] <= end_month)]

MONTH=2
LAST=28
YEAR=2021
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2102 = df_obs_2021[(df_obs_2021["datetime"] >= beg_month) & (df_obs_2021["datetime"] <= end_month)]

MONTH=3
LAST=31
YEAR=2021
from datetime import datetime
beg_month = datetime(YEAR,MONTH,1)
end_month = datetime(YEAR,MONTH,LAST)
obs_2103 = df_obs_2021[(df_obs_2021["datetime"] >= beg_month) & (df_obs_2021["datetime"] <= end_month)]


In [77]:
geo_obs_2102 = gpd.GeoDataFrame(
    obs_2102, geometry=gpd.points_from_xy(obs_2102.lon, obs_2102.lat), crs="EPSG:4326"
)
match_kas_2102 = gpd.sjoin(df_kas,geo_obs_2102,op = 'contains')
sel_data_2102=match_kas_2102[match_kas_2102["datetime"] == datetime(2021,2,9,22)]

sel_data_2102[["SID","lat","lon"]].drop_duplicates(subset=["SID"]).shape

include_these=sel_data_2102["SID"].to_list()
to_plot=geo_obs_2102[geo_obs_2102["SID"].isin(include_these)]
to_plot.drop_duplicates(subset=["SID"],inplace=True)

In [78]:
m = fl.Map(zoom_start=100, layers_control=True, tiles="OpenStreetMap")
for _, r in to_plot.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    geo_j.add_to(m)
   
for _, r in points_kastrup.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    geo_j.add_to(m)
m    

In [80]:
n2101=to_plot.shape[0]
print(f"stations in 2021, feb 29: {n2101}")

stations in 2021, feb 29: 27
